<a href="https://colab.research.google.com/github/hsnovasma091-svg/gldsky/blob/main/password_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import hashlib
import binascii
import secrets
import getpass
from pathlib import Path

# main.py
# VS Code üçün uyğunlaşdırılmış təhlükəsiz parol idarəetmə nümunəsi

PW_FILE = Path.home() / ".pw_hash"
ITERATIONS = 200_000
SALT_LEN = 16
KEY_LEN = 32
MAX_ATTEMPTS = 3


def hash_password(password: str, salt: bytes = None, iterations: int = ITERATIONS) -> str:
    if salt is None:
        salt = secrets.token_bytes(SALT_LEN)
    dk = hashlib.pbkdf2_hmac("sha256", password.encode("utf-8"), salt, iterations, dklen=KEY_LEN)
    return f"{binascii.hexlify(salt).decode()}${iterations}${binascii.hexlify(dk).decode()}"


def verify_password(stored: str, password: str) -> bool:
    try:
        salt_hex, iter_str, hash_hex = stored.split("$")
        salt = binascii.unhexlify(salt_hex)
        iterations = int(iter_str)
        expected = binascii.unhexlify(hash_hex)
        dk = hashlib.pbkdf2_hmac("sha256", password.encode("utf-8"), salt, iterations, dklen=len(expected))
        return secrets.compare_digest(dk, expected)
    except Exception:
        return False


def _prompt(prompt_text: str) -> str:
    try:
        return getpass.getpass(prompt_text)
    except Exception:
        # Fallback to input() if getpass is not supported (e.g., certain IDE consoles)
        return input(prompt_text)


def ensure_password_file(path: Path) -> None:
    if path.exists():
        return
    print("Parol təyin edilməyib. Yeni parol yaradın.")
    while True:
        p1 = _prompt("Yeni parol: ")
        p2 = _prompt("Parolu təsdiqləyin: ")
        if p1 != p2:
            print("Parollar uyğun deyil, yenidən cəhd edin.")
            continue
        if len(p1) < 6:
            print("Parol ən azı 6 simvol olmalıdır.")
            continue
        stored = hash_password(p1)
        path.write_text(stored, encoding="utf-8")
        # Try to restrict file permissions on POSIX systems
        try:
            path.chmod(0o600)
        except Exception:
            pass
        print("Parol uğurla yaradıldı.")
        break


def load_stored_password(path: Path) -> str:
    return path.read_text(encoding="utf-8").strip()


def change_password(path: Path) -> None:
    if not path.exists():
        print("Parol faylı tapılmadı.")
        return
    stored = load_stored_password(path)
    current = _prompt("Cari parol: ")
    if not verify_password(stored, current):
        print("Cari parol səhvdir.")
        return
    while True:
        p1 = _prompt("Yeni parol: ")
        p2 = _prompt("Parolu təsdiqləyin: ")
        if p1 != p2:
            print("Parollar uyğun deyil, yenidən cəhd edin.")
            continue
        if len(p1) < 6:
            print("Parol ən azı 6 simvol olmalıdır.")
            continue
        path.write_text(hash_password(p1), encoding="utf-8")
        try:
            path.chmod(0o600)
        except Exception:
            pass
        print("Parol dəyişdirildi.")
        break


def main():
    args = sys.argv[1:]
    if "--help" in args or "-h" in args:
        print("İstifadə:")
        print("  python main.py         # sistemə daxil olmaq")
        print("  python main.py --change  # parolu dəyişmək")
        return

    if "--change" in args:
        ensure_password_file(PW_FILE)
        change_password(PW_FILE)
        return

    ensure_password_file(PW_FILE)
    stored = load_stored_password(PW_FILE)

    for attempt in range(1, MAX_ATTEMPTS + 1):
        entered = _prompt("Sistemə daxil olmaq üçün parol daxil edin: ")
        if verify_password(stored, entered):
            print("✅ Parol doğru! Sistem başlayır...")
            # --- Sizin əsas kodunuz burada başlayır ---
            print("🎉 Sistemin əsas funksiyaları burada işləyir...")
            # burada kamera, üz tanıma və s. funksiyaları çağırın
            return
        else:
            remaining = MAX_ATTEMPTS - attempt
            print(f"❌ Səhv parol! Qalan cəhdlər: {remaining}")
            if remaining == 0:
                print("Sistem bağlanır.")
                sys.exit(1)


if __name__ == "__main__":
    main()

Parol təyin edilməyib. Yeni parol yaradın.
Yeni parol: ··········
Parolu təsdiqləyin: ··········
Parol ən azı 6 simvol olmalıdır.
Yeni parol: ··········
Parolu təsdiqləyin: ··········
Parol uğurla yaradıldı.
Sistemə daxil olmaq üçün parol daxil edin: ··········
✅ Parol doğru! Sistem başlayır...
🎉 Sistemin əsas funksiyaları burada işləyir...
